In [147]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import ttc
from pythainlp import word_tokenize, Tokenizer
from pythainlp import thai_digits, thai_letters
from pythainlp.util import isthaichar
import pickle as pkl

In [148]:
visited_url = set()
count = 0
def get_all_link(soup):
    all_link = soup.find_all('a')
    links = []
    for link in all_link:
        st = link.get('href')
        if st == None:
            continue
        if len(st) <= 10 or st == 'javascript:void(0);':
            continue
        if st[:2] == '//':
            st = st[2:]
        links.append(st)
    return links

def scrape_data(url="http://pantip.com", depth=3):
    global count, visited_url
    if depth == 0:
        return None
    if url[:4] != 'http':
        url = 'http://' + url
    #the python bugs if the url opens the pdf file (_markupbase Unbound error (match variable referenced before assignment))
    if url in visited_url or '.pdf' in url \
                          or 'pantip' not in url \
                          or 'pantipmarket' in url \
                          or '.gif' in url \
                          or '.jpg' in url \
                          or '.jpeg' in url:
        return None
    visited_url |= {url}
    try:
        html = urlopen(url)
    except:
        return None
    if html.getcode() != 200:
        print('error: GOT {} from url:{}'.format(html.getcode(),url))
        return None
    print('200 OK',url)
    count += 1
    soup = BeautifulSoup(html, 'html.parser')
    html.close()
    text = [st.strip() for st in soup.get_text().strip().lower().split('\n') if len(st.strip()) > 1 \
            and len({'{','}','#','$','(',')',',','/',';','\\',':'} & set(st.strip())) == 0]
    links = get_all_link(soup)
    
    if links != None:
        for url in links:
            _t = scrape_data(url, depth - 1)
            if _t != None:
                text += _t
    return text

In [149]:
extracted_data = scrape_data(depth=3)
print('scraped {} sites, got {} sentences of data'.format(count,len(extracted_data)))

200 OK http://pantip.com
200 OK http://pantipstore.com/
200 OK http://www.pantip.com
200 OK https://www.facebook.com/pantipdotcom/
200 OK https://l.facebook.com/l.php?u=https%3A%2F%2Fpantip.com%2F&h=AT1GC4YesWWPrz57QDiVginbgSOSiMxPeh5NPEtZ4b4lUNd7uZ3_-7JOGk_HW80DL0XpMvU7PRWDOtjeMhCqaOF3zI7Ou1SAfbMjFyeqmM95U7yYhIeuOZ3XSmDDtB4s3NGVoEd6cijkoJhc
200 OK https://l.facebook.com/l.php?u=https%3A%2F%2Fpantip.com%2F&h=AT0gFiTrN2_P0LupigV9t6WsPXMififQNSTXCakrCmFwgOk3weuMizwcA5HmN0Z6N-3Harif9_rPE10UFlxAOfGzjUy3qgv9zWeoe9X4wcrlC4GZBYr-2jlihRCkGLttWqrTt5L4udTg_z7Eo6k
200 OK https://web.facebook.com/pantipdotcom/likes?ref=page_internal
200 OK https://web.facebook.com/pantipdotcom/likes
200 OK https://web.facebook.com/pantipded/?ref=py_c


KeyboardInterrupt: 

In [136]:
extracted_data_2 = []
with open('true_wallet_scraped_list.pickle', 'rb') as handle:
    extracted_data_2 = pkl.load(handle)

In [141]:
extracted_data_2 = sorted(list(set(extracted_data_2)))

In [142]:
len(extracted_data_2)

7221

In [145]:
extracted_data_2[:200]

['"asia fitness conference 2019" งานประชุมเชิงปฏิบัติการด้านกีฬาและการออกกำลังกายระดับโลกที่ fitness professionals ทุกคนห้ามพลาด!',
 '"full house the special"',
 '"not rated"',
 '"paul" เฟรนช์เบเกอรี่พรีเมี่ยมระดับโลก',
 '"true venture fund" ที่จะช่วยสนับสนุนเงินทุน กว่า 30 ล้าน usd ในระยะเวลา 5 ปี',
 '"ชิตัง เม" แปลว่า อะไร ?  ความหมายแท้จริงตามพระไตรปิฏก คืออะไร ?',
 '"ซีพี" ร่วมกับ "แบงค็อก ยูไนเต็ด"',
 '"ทรูช้อปปิ้ง" คัดสรรสินค้าและบริการต่างๆ ที่มีคุณภาพ นำเสนอต่อลูกค้าทั่วประเทศ เพื่อให้สามารถเลือกซื้อสินค้าด้วยความสะดวกสบาย โดยมีสินค้าหลากหลาย ครอบคลุมทุกประเภท ไม่ว่าจะเป็น สินค้าแฟชั่นและความงาม เสื้อผ้า เครื่องประดับ เครื่องสำอางค์ สินค้าในครัวเรือน ของตกแต่งบ้าน เครื่องออกกำลังกาย เครื่องใช้ในครัว เครื่องใช้ไฟฟ้าและอิเล็กทรอนิกส์ในชีวิตประจำวัน โทรศัพท์มือถือ แท็ปแล็ต เครื่องใช้ไฟฟ้าในครัว รวมทั้ง แพ็กเกจทัวร์ เพื่อให้สามารถตอบสนองทุกความต้องการในทุกไลฟ์สไตล์ของลูกค้า',
 '"บาบา บีช คลับ ภูเก็ต" โครงการที่พักอาศัยและโรงแรมในสไตล์บีชคลับระดับลักซ์ชัวรี่แห่งใหม่ล่าสุดในภูเก็ต ตั

In [ ]:
def preprocess_sentence(sen):
    n = len(sen)
    st = ""
    for i in range(n):
        if i + 1 < n:
            if not isthaichar(sen[i]) and isthaichar(sen[i + 1]):
                st += sen[i] + ' '
            else:
                st += sen[i]
